In [1]:
import datetime
import pytz
import grpc
import baseline_optimizer_pb2
import baseline_optimizer_pb2_grpc
import xbos_services_getter as xsg

In [2]:
XBOS_MICROSERVICES_HOST_ADDRESS="ms.xbos.io:9001" 
temperature_bands_stub = xsg.get_temperature_band_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
occupancy_stub = xsg.get_occupancy_stub(XBOS_MICROSERVICES_HOST_ADDRESS)
building_zone_names_stub = xsg.get_building_zone_names_stub(XBOS_MICROSERVICES_HOST_ADDRESS)

channel = grpc.insecure_channel("localhost:50050")
baseline_optimizer_stub = baseline_optimizer_pb2_grpc.BaselineOptimizerStub(channel)

In [3]:
building = "ciee"
zones = xsg.get_zones(building_zone_names_stub,building)
start = pytz.timezone("US/Pacific").localize(datetime.datetime(year=2019, month=6, day=5, hour=9, minute=0))
end = start + datetime.timedelta(hours=1)
start = start.replace(microsecond=0)
end = end.replace(microsecond=0)
start_unix = int(start.timestamp() * 1e9)
end_unix = int(end.timestamp() * 1e9)

window = "1h"
unit = "F"
occupancy = False
do_not_exceed = True
max_zones = int(len(zones)/2)
include_all_zones = True
starting_temperatures = {}
expansion_degrees = {}
comfort_band = {}
occupancy_prop = {}
do_not_exceed_band = {}

t = 84
for zone in zones:
    t += 1
    starting_temperatures[zone] = t
    expansion_degrees[zone] = 10.0
    comfort_band[zone] = xsg.get_comfortband(temperature_bands_stub,building,zone,start,end,window).iloc[0]
    occupancy_prop[zone] = xsg.get_occupancy(occupancy_stub,building,zone,start,end,window).iloc[0]
    do_not_exceed_band[zone]= xsg.get_do_not_exceed(temperature_bands_stub,building,zone,start,end,window).iloc[0]
print(comfort_band)
print(do_not_exceed_band)
print(occupancy_prop)
normal_actions = baseline_optimizer_stub.GetNormalScheduleAction(baseline_optimizer_pb2.NormalScheduleRequest(building=building,zones=zones,start=start_unix,end=end_unix,window=window,starting_temperatures=starting_temperatures,unit=unit,occupancy=occupancy,do_not_exceed=do_not_exceed))
print("normal",normal_actions.actions)
expansion_actions = baseline_optimizer_stub.GetSetpointExpansionAction(baseline_optimizer_pb2.SetpointExpansionRequest(building=building,zones=zones,start=start_unix,end=end_unix,window=window,starting_temperatures=starting_temperatures,unit=unit,occupancy=occupancy,do_not_exceed=do_not_exceed,expansion_degrees=expansion_degrees))
print("expansion",expansion_actions.actions)
demand_charge_actions = baseline_optimizer_stub.GetDemandChargeAction(baseline_optimizer_pb2.DemandChargeRequest(building=building,zones=zones,start=start_unix,end=end_unix,window=window,starting_temperatures=starting_temperatures,unit=unit,occupancy=occupancy,do_not_exceed=do_not_exceed,max_zones=max_zones,include_all_zones=include_all_zones))
print("demand",demand_charge_actions.actions)

{'hvac_zone_centralzone': t_high    78
t_low     74
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object, 'hvac_zone_eastzone': t_high    78
t_low     74
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object, 'hvac_zone_northzone': t_high    78
t_low     74
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object, 'hvac_zone_southzone': t_high    78
t_low     74
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object}
{'hvac_zone_centralzone': t_high    86
t_low     54
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object, 'hvac_zone_eastzone': t_high    86
t_low     54
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object, 'hvac_zone_northzone': t_high    86
t_low     54
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object, 'hvac_zone_southzone': t_high    86
t_low     54
unit       F
Name: 2019-06-05 09:00:00-07:00, dtype: object}
{'hvac_zone_centralzone': occupancy    1.0
Name: 2019-06-05 09:00:00-07:00, dtype: float64, 'hvac_zone_eastzone': oc